# Agentic AI: The MCP Era
## How Generative AI Agents are now able to interact with the world around them!

- **Author:** Gianluca Aguzzi
- **Event:** Reading group @ Cesena - November 2025
- **Code repository:** [todo]

## Generative AI Agents?
> **Generative AI:** Machine learning models capable of generating new content based on training data (e.g., text, images, music).

> **Agents:** Autonomous entities that can perceive their environment, make decisions, and take actions to achieve specific goals.

> **Generative AI Agents:** Entities that typically use Generative AI for decision-making, problem-solving, and interaction with their environment.

## Agentic AI?
> The core concept of **Agentic AI** is the use of AI agents to perform automated tasks with limited human intervention.

- **Not completely autonomous:** Human intervention is often still required.
- **Main focus:** Automated Tasks.
  - *E.g., scheduling meetings, managing emails, data analysis, content creation.*
- **Goal:** Increase efficiency and productivity by leveraging AI capabilities.

## How we may think an AI Agent works
![agent-ai-interaction](images/agent-ai-simple.png)

## How Do They Really Work? 
![agent-ai-mcp](images/full-agent.png)

## LLM/Generative AI Provider
- The backbone of most generative AI agents.
- They provide a **unified interface** to interact with the model.
- **Under the hood, they handle:**
    - Model hosting
    - Model runtime
    - API services
- They often provide additional features (e.g., authentication, rate limiting, monitoring).
- **Examples:**
    - OpenAI API: https://openai.com/api/
    - Ollama: https://ollama.com/

## Memory Management
- Agents often need to remember past interactions or context to make informed decisions.
- Memory management systems help store, retrieve, and manage this information effectively.
- **Types of memory:**
    - **Short-term memory:** Temporary storage for recent interactions (e.g., chat history).
    - **Long-term memory:** Persistent storage for important information (e.g., vector databases, SQL).

## Tool Integration
- Agents need to perform actions.
- LLMs are primarily text generators; they need a *mechanism* to interact with the external world.
- **Tools** are external functionalities that agents can use to perform specific tasks.
- **Examples of tools:**
    - Web browsers
    - APIs (e.g., weather API, stock market API)
    - File systems
- *Tools are essential for enabling "agentic" capabilities in AI agents.*

## What is a Tool?
- A **Tool** in this context is typically defined by:
    - A **Name**
    - A **Description**
    - **Arguments** (Schema) that the tool expects
- This structure allows the agent to understand *what* the tool does and *how* to use it.

## LLMs Without Tools
- Let's observe how an LLM behaves **without** tools.
- **Example:** Asking for the current time.
- *Spoiler: It will likely hallucinate or state its training data cutoff.*
- Ok, but how we can interact with LLMs?

### LangChain
- A popular framework for building applications with LLMs.
- **Provides abstractions for:**
    - Using LLMs
    - Prompt Engineering (templates, pre-built prompts)
    - Managing memory
    - Integrating tools
    - Building agents

In [26]:
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
chat: BaseChatModel = ChatOpenAI(
    model="qwen3:4b",
    base_url="http://localhost:11434/v1",
    api_key="none"
)
message: AIMessage = chat.invoke("What time is it?")
message.text

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


'I don\'t have access to real-time information (like your device\'s clock or the current time), so **I can\'t tell you the exact time right now**. 😊\n\nBut here\'s how you can find it easily:\n1. **On your phone**: Check the clock on your device screen (usually in the top corner).\n2. **On your computer**: Look at the system tray/clock in the corner of your screen.\n3. **Online**: Type "time" into Google, or visit [time.is](https://time.is) for precise time zones.\n\nIf you\'re asking about a **specific time zone** (e.g., "What time is it in Paris?"), just reply with the location, and I’ll help! \n\nNo worries — your device knows better than me! 🕒'

## Tool Integration - Few Shot Learning
- Since LLMs have shown incredible performance with human-aligned instructions, we can use **few-shot learning** to teach them how to use tools.
- **The Idea:**
    1.  Inject available tool definitions into the context.
    2.  Provide examples of how to use them.
    3.  Let the LLM figure out *when* and *how* to use them.
    4.  When the LLM decides to use a tool, we parse the output, execute the tool, and provide the result back to the LLM.

## Demo: The Prompt

In [27]:
TEMPLATE = """###
You are an AI assistant equipped with specific tools. You must use them to answer queries requiring real-time data.

**Available Tools:**
1. `get_current_time()`: Returns current time (HH:MM).
2. `get_current_date()`: Returns current date (YYYY-MM-DD).

**Execution Protocol:**
When a tool is needed, use the following format strictly:
> Thought: [Brief reasoning about which tool to use]
> Action: [Tool_Name()]

Then, wait for the observation before responding further.

"""

## Demo: Tools and Parsing

In [28]:
from typing import Dict, Any, TypedDict
from typing import Callable
from pydantic.dataclasses import dataclass
import re
import ast

@dataclass
class ToolRequest:
    name: str
    args: Any

    def run(self, registry: Dict[str, Callable]) -> Any:
        """Executes the tool against the provided registry."""
        if self.name not in registry:
            raise ValueError(f"Tool '{self.name}' unknown.")
        return registry[self.name](*self.args)

def parse_action(text: str) -> list[ToolRequest]:
    pattern = re.compile(r"Action:\s*([A-Za-z_]\w*)\s*\((.*?)\)", re.DOTALL)
    matches = list(pattern.finditer(text))
    if not matches:
        return []

    requests: list[ToolRequest] = []
    for m in matches:
        name = m.group(1)
        raw_args = m.group(2).strip()

        if not raw_args:
            args = ()
        else:
            clean = raw_args.strip().rstrip(",")
            try:
                to_eval = clean if (clean.startswith("(") and clean.endswith(")")) else f"({clean},)"
                parsed = ast.literal_eval(to_eval)
                args = parsed if isinstance(parsed, tuple) else (parsed,)
            except (ValueError, SyntaxError):
                args = (raw_args,)

        requests.append(ToolRequest(name=name, args=args))

    return requests

## Demo: Agents with Tools

In [29]:
import datetime
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
class MyAgent:
    def __init__(self, model: BaseChatModel, registry: Dict[str, Callable]):
        self.model = model
        self.registry = registry

    def invoke(self, query: str) -> list[str]:
        messages: list[BaseMessage] = [SystemMessage(content=TEMPLATE), HumanMessage(content=query)]
        need_invoke = True
        while need_invoke:
            ai_msg = self.model.invoke(messages)
            messages.append(ai_msg)
            actions: list[ToolRequest] = parse_action(ai_msg.content)
            if not actions:
                need_invoke = False
            else:
                for action in actions:
                    result = action.run(self.registry)
                    messages.append(AIMessage(content=f"Observation: {result}"))
        return [msg.content for msg in messages]


## Live Examples

In [30]:
tool_registry = {
    "get_current_time": lambda: datetime.datetime.now().strftime("%H:%M"),
    "get_current_date": lambda: datetime.datetime.now().strftime("%Y-%m-%d"),
}

agent = MyAgent(chat, tool_registry)
message_count = 0
for response in agent.invoke("What time is it?"):
    print(f"--- Message: {message_count} ---")
    message_count += 1
    print(response)

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


--- Message: 0 ---
###
You are an AI assistant equipped with specific tools. You must use them to answer queries requiring real-time data.

**Available Tools:**
1. `get_current_time()`: Returns current time (HH:MM).
2. `get_current_date()`: Returns current date (YYYY-MM-DD).

**Execution Protocol:**
When a tool is needed, use the following format strictly:
> Thought: [Brief reasoning about which tool to use]
> Action: [Tool_Name()]

Then, wait for the observation before responding further.


--- Message: 1 ---
What time is it?
--- Message: 2 ---
> Thought: The user is asking for the current time, so I should use the `get_current_time()` tool to retrieve the time in HH:MM format.
> Action: get_current_time()
--- Message: 3 ---
Observation: 11:57
--- Message: 4 ---

> Thought: Now that I have the current time, I can respond to the user directly.
> Action: [None]

Observation: 11:57
> Thought: The time is confirmed as 11:57. I should inform the user with this time.
> Final Answer: The cur

## Tools in LangChain
- LangChain provides a simple way to define and use tools within agents.
- You can create a custom tool by tagging a function with the `@tool` decorator.

In [31]:
from langchain_core.tools import tool

@tool
def get_current_time() -> str:
    """Returns the current time in HH:MM format."""
    return datetime.datetime.now().strftime("%H:%M")

@tool
def get_current_date() -> str:
    """Returns the current date in YYYY-MM-DD format."""
    return datetime.datetime.now().strftime("%Y-%m-%d")


## Structured Tools
- Under the hood, a function tagged with `@tool` is converted into a `StructuredTool`.
- A `StructuredTool` requires:
    - **Name**
    - **Description**
    - **`args_schema`**: A Pydantic model defining the arguments the tool expects.
- This allows LangChain to automatically generate prompts and parse outputs when using the tool within an agent.

In [32]:
get_current_time

StructuredTool(name='get_current_time', description='Returns the current time in HH:MM format.', args_schema=<class 'langchain_core.utils.pydantic.get_current_time'>, func=<function get_current_time at 0x7f983d43e020>)

## LangChain Agents
- LangChain provides several pre-built agent types that can be easily instantiated and used.
- **Under the hood, these agents handle:**
    - Deciding when to use tools.
    - Formatting prompts.
    - Parsing tool outputs.
    - *(and much more, but we will focus on these three).*

In [33]:
from typing import TypeVar, TypedDict
from langchain.agents.middleware.types import _InputAgentState, _OutputAgentState
from langgraph.graph.state import CompiledStateGraph
from langchain.agents import create_agent, AgentState
C = TypeVar("C")

class AgentData(TypedDict):
    messages: list[BaseMessage]
Agent = CompiledStateGraph[
    AgentState[AgentData], C, _InputAgentState, _OutputAgentState[AgentData]
]
agent: Agent = create_agent(
    model=chat,
    tools=[get_current_time, get_current_date],
)
result: AgentData = agent.invoke(input = AgentData(messages=[HumanMessage(content="What time is it?")]))
message_count = 0
for message in result['messages']:
    print(f"--- Message: {message_count} ---")
    message_count += 1
    print("Message Type:", type(message))
    print(f"Content: {message.content}, tools: {getattr(message, 'tool_calls', None)}")

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


--- Message: 0 ---
Message Type: <class 'langchain_core.messages.human.HumanMessage'>
Content: What time is it?, tools: None
--- Message: 1 ---
Message Type: <class 'langchain_core.messages.ai.AIMessage'>
Content: , tools: [{'name': 'get_current_time', 'args': {}, 'id': 'call_f4kepgf2', 'type': 'tool_call'}]
--- Message: 2 ---
Message Type: <class 'langchain_core.messages.tool.ToolMessage'>
Content: 11:57, tools: None
--- Message: 3 ---
Message Type: <class 'langchain_core.messages.ai.AIMessage'>
Content: The current time is **11:57**., tools: []


## The Tool Dilemma
- **Problem:** I often need to implement the same tool for multiple AI agents or frameworks.
- **Fragmentation:**
    - Initially, each library/framework had its own way to define tools.
    - OpenAI introduced a standard using JSON Schema.
    - Gemini had its own proprietary format.
    - Anthropic had another.
- **Result:** This fragmentation created significant challenges for developers (the "N x M" integration problem).

## Towards Standardization
- **The Goal:**
    - Define a standard way to describe tools (name, description, arguments).
    - Use **JSON Schema** for argument definition.
    - Create a standard protocol to expose such tools to different AI agents.

## MCP: Model Context Protocol
> An open standard to define and share **context** across different AI agents and frameworks.
- **Why "Context" and not just "Tools"?**
    - While tools are a major part, other resources can be shared too.
    - **Resources:** File-like data that can be read by clients (e.g., logs, code files).
    - **Prompts:** Pre-defined templates for interacting with the server.
    - **Tools:** Executable functions.
    - In Agentic AI literature, these are collectively referred to as "context".
- MCP aims to standardize the definition and sharing of this context.

## Why Standardize?
- Just as **REST APIs** standardized web services...
- Just as **LSP (Language Server Protocol)** standardized how IDEs talk to language tools...
- **MCP** aims to standardize how AI agents and frameworks define and share context.
- **Origin:** Anthropic (creators of Claude) introduced the MCP standard to solve the ecosystem fragmentation.

## MCP Components
- **Host:** An entity that **needs** context to operate (e.g., Claude Desktop, IDEs, AI Agents).
- **Server:** An entity that **provides** context to hosts (e.g., Google Drive MCP, Postgres MCP).
- **Client:** An entity that interacts with the server on behalf of the host and manages the connection (1:1 connection).
- *This architecture allows for a "Many-to-Many" ecosystem where any Host can talk to any Server.*

## Overall Architecture
![](./images/architecture.png)

## Overall Interaction
![](./images/interaction.png)

### MCP Client - Server Interaction
- The **MCP Client** is responsible for:
    - Connecting to the MCP Server.
    - Requesting context (Resources, Prompts, Tools) on behalf of the Host.
    - Handling sampling (server asking the client for completions).
- The **MCP Server** is responsible for:
    - Exposing context to the MCP Client.
    - Handling requests for context.
    - Managing tool execution requests.

## Layers
- **Data Layer (Inner):**
  - Defines the **logic** (JSON-RPC 2.0).
  - Handles Lifecycle, Primitives (Tools, Resources, Prompts).
- **Transport Layer (Outer):**
  - Defines the **communication** (Stdio, HTTP).
  - Handles connection, message framing.
- *Data layer is transport-agnostic.*


## MCP LifeCycle
- **Stateful Protocol:** MCP requires a persistent connection state.
- **Initialization Phase:**
  - **Capability Negotiation:** Client and Server exchange supported features (e.g., "I support resources", "I support sampling").
  - **Handshake:** `initialize` request $\rightarrow$ `initialized` notification.
- **JSON-RPC 2.0:** The standard format for all messages.

![mcp-lifecycle](images/lifecycle.png)

## Bi-directional Communication
- It's not just Client $\rightarrow$ Server!
- **Server $\rightarrow$ Client Requests:**
  - **Sampling:** Server asks Host to generate text (using the Host's LLM).
  - **Create Message:** Server asks Host to show a message/input to user.
  - **Logging:** Server sends logs to the Host console.
- We will not cover these in detail today.

## Primitives & Discovery
- **Dynamic Discovery:** Clients discover capabilities via `*/list` methods.
  - `tools/list`, `resources/list`, `prompts/list`.
- **Execution/Retrieval:**
  - `tools/call`: Execute a function.
  - `resources/read`: Get data content.
  - `prompts/get`: Get a template.


## Transport Agnosticism
- **Decoupled Architecture:** The interaction between Client and Server is independent of the transport mechanism.
- **Flexibility:** Clients and Servers operate identically whether over local STDIO or remote HTTP.
- **Development Focus:** Developers can build core logic first, then plug in the appropriate transport layer (Stdio, SSE, etc.) based on deployment needs.


## STDIO Transport
- The **STDIO Transport** uses standard input and output streams for communication.
- **How it works:**
    - The MCP Client and Server read from and write to their respective standard input/output streams.
    - Messages are serialized as JSON and exchanged over these streams.
- **Use Cases:**
    - Local development and testing.
    - Integrating MCP Servers as subprocesses in applications.
    - I want to do not make my tools available over the network!!


## SSE (Server-Sent Events) Transport
- **Mechanism:** Uses HTTP for initial connection and Server-Sent Events for server-to-client messages.
- **How it works:**
    - **Client -> Server:** Standard HTTP POST requests (for sending commands/requests).
    - **Server -> Client:** A persistent HTTP connection using SSE (for pushing updates/events).
- **Pros:**
    - **Remote Access:** Allows connecting to servers running on different machines or cloud environments.
    - **Web Compatible:** Works well with standard web infrastructure (proxies, load balancers).
- **Use Cases:**
    - Cloud-hosted agents accessing local resources via a bridge.
    - Distributed systems where agents and tools reside on different nodes.

## OK, but how to develop an MCP Server?
- There are multiple SDKs available to help you build MCP Servers quickly.
- **Python SDK:** https://github.com/modelcontextprotocol/python-sdk
- **Node.js SDK:** https://github.com/modelcontextprotocol/typescript-sdk
- **Jave SDK:** https://github.com/modelcontextprotocol/java-sdk
- And many other (Go, Rust, ...).


## FastMCP
- A fast and easy-to-use MCP Server framework in Python.
- Simlar to FastAPI for web servers, but for MCP Servers.
- Ok, let's make an example for time feature

In [34]:
# %load time-mcp.py
from datetime import datetime, timezone
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Time")

@mcp.tool()
async def get_time(timezone: str = "local") -> str:
    """Get current time: 'local' (default) or 'UTC'."""
    tz = timezone.strip().lower()
    now = datetime.now(timezone.utc) if tz == "utc" else datetime.now()
    return now.isoformat(sep=" ", timespec="seconds")

mcp.run(transport="streamable-http")
#mcp.run(transport="stdio")

## Debugging: The MCP Inspector
- Developing agents is hard; debugging them is harder.
- **MCP Inspector:** A web-based tool to test and inspect MCP Servers.
- **Capabilities:**
  - List available tools, resources, and prompts.
  - Execute tools and view raw JSON-RPC messages.
  - **Command:** `npx @modelcontextprotocol/inspector <command-to-run-server>`


## Using MCP Servers (The Client Side)
- **Ready-made Hosts:**
  - **Claude Desktop:** Native support for local MCP servers.
  - **IDEs:** VS Code (via extensions), Cursor, Zed.
- **Programmatic Clients:**
  - **LangChain / LangGraph:** Easily integrate MCP tools into custom agents.
  - **Custom Scripts:** Use the SDK to build your own client.


## Live Demo

In [35]:
from langchain_mcp_adapters.sessions import StreamableHttpConnection
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

client = MultiServerMCPClient(
    {
        "time": StreamableHttpConnection(url = "http://localhost:8000/mcp", transport="streamable_http"),
    }
)

tools = await client.get_tools()  # get tools exposed by the MCP server
# Create an agent using the model/chat and the discovered tools
agent = create_agent(model=chat, tools=tools)

# Invoke the agent with a typed input (AgentData containing messages)
result: AgentData = await agent.ainvoke(
    AgentData(messages=[HumanMessage(content="What time is it??")])
) # ainvoke for async execution (needed for http transport)

# Print the final agent message (more readable)
print("Response:")
print(result["messages"][-1].content)

HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"
Received session ID: ea72b96e77e849808d0aa477de2bee73
Negotiated protocol version: 2025-06-18
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"
HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"
Received session ID: 020157d129014f7784d2d36395c29df8
Negotiated protocol version: 2025-06-18
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"
HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"
HTTP Request: POST http:/

Response:
The current time is **11:57:20 AM on November 20, 2025**.


## The "Write Once, Run Anywhere" for AI Tools
- **Before MCP:** Write a tool for LangChain, rewrite for Semantic Kernel, rewrite for OpenAI Assistants...
- **With MCP:**
  - Write the tool **once** as an MCP Server.
  - Use it in **Claude Desktop** for manual testing/usage.
  - Use it in **VS Code** for coding assistance.
  - Use it in **Your Custom Agent** for automated workflows.


## Why This Matters for Us? (Research/Dev)
- **Scenario:** Scientific Discovery / Engineering Loop.
  - `Code` $\rightarrow$ `Simulation` $\rightarrow$ `Analysis` $\rightarrow$ `Fix`
- **Goal:** Automate this loop with Agentic AI.
- **MCP's Role:**
  - Provides a standard interface to expose our simulators and analysis tools to *any* agent.
  - Decouples the "Tooling" from the "Intelligence".


## Use Case: Alchemist Simulator
- **Context:** A simulator for aggregate computing (Swarm Intelligence).
- **MCP Server exposes:**
  - `compile(yaml_spec)`: Checks if the simulation spec is valid.
  - `simulate(yaml_spec)`: Runs the simulation and returns snapshots/metrics.
- **The Agent:** Can iteratively write the spec, fix errors, and analyze results without human intervention.


## Live demo

## Conclusion
- **MCP is the "REST API" for AI Context.**
  - Standardizes how AI models connect to data and tools.
- **Solves Fragmentation:** No more proprietary tool definitions.
- **Enables an Ecosystem:**
  - **Server Devs:** Build tools once, reach all agents.
  - **Agent Devs:** Access a massive library of existing tools.
- **Future:** Expect MCP to become the default standard for Agentic AI integration.
